<a href="https://colab.research.google.com/github/salixkang/COSE474_Final/blob/main/Final_Project_2018320223_%EA%B0%95%EC%8A%B9%EB%AA%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final Project 2018320223 컴퓨터학과 강승모

In [ ]:
import numpy as np
import random

from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class TileGameEnv:
    def __init__(self, layout, max_card_usage, max_swap):
        self.initial_layout = np.array(layout)
        self.layout = None
        self.max_card_usage = max_card_usage
        self.total_normal_card_types = 10
        self.max_enhancement = 2
        self.max_swap = max_swap
        self.action_space = 4  # 0: Use Card 1, 1: Use Card 2, 2: Swap Card 1, 3: Swap Card 2
        self.tile_types = ['empty', '정상', '왜곡', '재배치', '축복', '추가', '강화', '복제', '신비']
        self.special_tile_effects = {
            '재배치': self._effect_rearrange,
            '축복': self._effect_blessing,
            '추가': self._effect_extra,
            '강화': self._effect_enhance,
            '복제': self._effect_clone,
            '신비': self._effect_mystery
        }
        self.card_types = ['충격파', '낙뢰', '용오름', '해일', '지진', '폭풍우', '대폭발', '정화', '벼락', '업화', '분출', '세계수의 공명']
        self.card_effects = {
            '충격파': self._effect_shockwave,
            '낙뢰': self._effect_thunder,
            '용오름': self._effect_dragon_rise,
            '해일': self._effect_tidal_wave,
            '지진': self._effect_earthquake,
            '폭풍우': self._effect_storm,
            '대폭발': self._effect_explosion,
            '정화': self._effect_purification,
            '벼락': self._effect_lightning,
            '업화': self._effect_upfire,
            '분출': self._effect_eruption,
            '세계수의 공명': self._effect_world_tree_resonance
        }
        self.current_special_tile = None  # 현재 스텝에서 생성된 특수 타일의 위치
        self.hand = []
        self.reserve = []
        self.hand_enhancements = [0, 0]
        self.cards_used = 0
        self.swap_count = 0
        self.used_card_index = None
        self.reset()

    def reset(self):
        # 게임 초기화
        self.layout = np.copy(self.initial_layout)
        self.hand = [random.randint(0, self.total_normal_card_types - 1) for _ in range(2)]  # 두 장의 패
        self.reserve = [random.randint(0, self.total_normal_card_types - 1) for _ in range(3)]  # 세 장의 다음 카드
        self.hand_enhancements = [0, 0]  # 패에 있는 각 카드의 강화 상태
        self.cards_used = 0
        self.swap_count = 0
        self.used_card_index = None
        self.current_special_tile = None
        return self._get_state()

    def step(self, action, position):
        assert action in range(self.action_space), "Invalid action!"
        reward = 0

        # Action: Use Card 1
        if action == 0:
            self.used_card_index = 0
            self._play_card(0, position)
            if self.current_special_tile:
                self.layout[self.current_special_tile] = 1
                self._generate_special_tile()

        # Action: Use Card 2
        elif action == 1:
            self.used_card_index = 1
            self._play_card(1, position)
            if self.current_special_tile:
                self.layout[self.current_special_tile] = 1
                self._generate_special_tile()

        # Action: Swap Card 1
        elif action == 2:
            self._swap_card(0)

        # Action: Swap Card 2
        elif action == 3:
            self._swap_card(1)

        done, result = self._is_game_over()
        if result == "Win":
            reward = 1
        elif result == "Lose":
            reward = -1

        return self._get_state(), reward, done, {}

    def _get_state(self):
        return np.array(self.layout.flatten().tolist() + self.hand + self.reserve + self.hand_enhancements)

    # 선택된 카드를 사용하는 메소드
    def _play_card(self, card_index, position):
        card_type = self.card_types[self.hand[card_index]]

        card_effect = self.card_effects.get(card_type, None)

        if card_effect:
            card_effect(position, self.hand_enhancements[card_index])

        self.hand[card_index] = self.reserve.pop(0)  # 예비 카드로 교체
        self.hand_enhancements[card_index] = 0  # 사용된 카드 강화 상태 리셋

        self.reserve.append(self._draw_new_card())  # 예비 카드 보충
        self.cards_used += 1 # 카드 사용 횟수 증가

        # 카드가 강화 가능한지 체크
        self._check_enhancement(card_index)

    # 카드를 교체하는 메소드
    def _swap_card(self, card_index):
        if self.swap_count < self.max_swap:
            self.swap_count += 1  # 교체 횟수 증가
            self.hand[card_index] = self.reserve.pop(0)  # 예비 카드로 교체
            self.hand_enhancements[card_index] = 0  # 사용된 카드 강화 상태 리셋
            self.reserve.append(self._draw_new_card())  # 예비 카드 보충
            self._check_enhancement(card_index)  # 강화 로직 적용

    # 새로운 카드를 예비 카드에 추가하는 메소드
    def _draw_new_card(self):
        return random.randint(0, self.total_card_types - 1)

    def _check_enhancement(self, new_card_index):
        other_card_index = 1 - new_card_index
        if self.hand[new_card_index] == self.hand[other_card_index]:
            if self.hand_enhancements[other_card_index] < self.max_enhancement:
                self.hand_enhancements[other_card_index] += 1  # 카드 강화
                self.hand[new_card_index] = self.reserve.pop(0)  # 다음 예비 카드를 패로 이동
                self.hand_enhancements[new_card_index] = 0
                self.reserve.append(self._draw_new_card())  # 예비 카드 보충
                self._check_enhancement(new_card_index)

    # 타일 파괴
    def _destroy_tile(self, position, enhancement=0, special=0, ):
        x, y = position
        tile_type = self.tile_types[self.layout[x, y]]

        if tile_type == '정상':
            self.layout[x, y] = 0
        elif tile_type == '왜곡':
            self._destroy_distorted_tile(position, enhancement, special)
        elif tile_type == 'empty':
            self.layout[x, y] = 0
        else:
            special_tile_effect = self.special_tile_effects.get(tile_type, None)
            self.layout[x, y] = 0
            self.current_special_tile = None
            if special_tile_effect:
                special_tile_effect()

    # 특정 위치가 게임판 내에 있는지 확인하는 메소드
    def _is_within_bounds(self, position):
        x, y = position
        return 0 <= x < self.layout.shape[0] and 0 <= y < self.layout.shape[1]

    # 랜덤 위치에 정상 타일을 생성하는 메소드
    def _create_random_tiles(self, count):
        for _ in range(count):
            empty_positions = [(i, j) for i in range(self.layout.shape[0])
                               for j in range(self.layout.shape[1]) if self.layout[i][j] == 0]
            if empty_positions:
                x, y = random.choice(empty_positions)
                self.layout[x, y] = 1  # 정상 타일 생성

    # 왜곡된 타일을 파괴했을 때 정상 타일을 생성하는 메서드
    def _destroy_distorted_tile(self, position, enhancement, special = 0):
        # 파괴할 위치에 왜곡된 타일이 있는지 확인
        if self.layout[position] == 2:
            self.layout[position] = 0  # 왜곡된 타일 파괴

            if enhancement != 2 or special == 1: # 강화 단계가 2거나 특수한 상황이라면 페널티 발생하지 않음
                # 타일이 없는 위치 중 무작위로 세 개의 위치에 정상 타일 생성
                empty_positions = np.argwhere(self.layout == 0)
                if empty_positions.size > 0:
                    for _ in range(min(3, len(empty_positions))):  # 최대 3개 혹은 가능한 개수만큼
                        x, y = empty_positions[random.randint(0, len(empty_positions) - 1)]
                        self.layout[x, y] = 1  # 정상 타일 생성

    # 남은 정상 타일 중 하나에 무작위 특수 타일 생성 메소드
    def _generate_special_tile(self):
        # 이전 특수 타일 위치 초기화
        self.current_special_tile = None
        normal_tiles = list(zip(*np.where(self.layout == 1)))
        if normal_tiles:  # 정상 타일이 존재하는 경우
            x, y = random.choice(normal_tiles)
            special_tile_type = random.randint(3, len(self.tile_types) - 1)
            self.layout[x, y] = special_tile_type
            self.current_special_tile = (x, y)  # 생성된 특수 타일의 위치 저장

    # 충격파
    def _effect_shockwave(self, position, enhancement):
        # 중심 위치의 타일은 100% 확률로 파괴
        center_x, center_y = position
        self._destroy_tile((center_x, center_y), enhancement)

        # 3x3 영역의 타일을 파괴하는 로직
        for i in range(center_x - 1, center_x + 2):
            for j in range(center_y - 1, center_y + 2):
                # 배열의 범위를 벗어나지 않는지 체크
                if self._is_within_bounds((i, j)):
                    # 중심 타일이 아닌 경우에만 확률 체크
                    if (i, j) != position:
                        if random.random() < 0.75 or enhancement > 0: # 75% or 강화되면 파괴
                            self._destroy_tile((i, j))

    # 낙뢰
    def _effect_thunder(self, position, enhancement):
        # 중심 위치는 항상 파괴
        center_x, center_y = position
        self._destroy_tile((center_x, center_y), enhancement)

        # 십자가 형태로 영향을 주므로 상하좌우 타일 파괴
        offsets = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dx, dy in offsets:
            x, y = center_x + dx, center_y + dy
            if self._is_within_bounds((x, y)):
                if random.random() < 0.5 or enhancement > 0:  # 50% 확률 or 강화되면 파괴
                    self._destroy_tile((x, y), enhancement)

    # 용오름
    def _effect_dragon_rise(self, position, enhancement):
        # 중심 위치는 항상 파괴
        center_x, center_y = position
        self._destroy_tile((center_x, center_y), enhancement)

        # X 자 형태로 영향을 주므로 대각선 타일 파괴
        diagonals = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
        for dx, dy in diagonals:
            x, y = center_x + dx, center_y + dy
            if self._is_within_bounds((x, y)):
                if random.random() < 0.5 or enhancement > 0:  # 50% 확률로 or 강화되면 파괴
                    self._destroy_tile((x, y), enhancement)

    # 해일
    def _effect_tidal_wave(self, position, enhancement):
        # 중심 위치는 100% 확률로 파괴
        center_x, center_y = position
        self._destroy_tile((center_x, center_y), enhancement)

        # 파괴 확률을 위한 기본값 설정
        base_prob = 1.0
        decrease_step = 0.15

        # 십자가 형태로 타일을 파괴
        # 상하 방향
        for i in range(self.layout.shape[0]):
            if i != center_x:  # 중심 위치 제외
                destroy_prob = base_prob - decrease_step * abs(i - center_x)
                if random.random() < destroy_prob or enhancement > 0:
                    self._destroy_tile((i, center_y), enhancement)

        # 좌우 방향
        for j in range(self.layout.shape[1]):
            if j != center_y:  # 중심 위치 제외
                destroy_prob = base_prob - decrease_step * abs(j - center_y)
                if random.random() < destroy_prob or enhancement > 0:
                    self._destroy_tile((center_x, j), enhancement)

    # 지진
    def _effect_earthquake(self, position, enhancement):
        # '지진' 카드: 시전 위치를 중심으로 가로 일렬 범위 모든 타일 타격
        for j in range(self.layout.shape[1]):
            destroy_prob = 1.0 - 0.15 * abs(j - position[1])
            if random.random() < destroy_prob or enhancement > 0:
                self._destroy_tile((position[0], j), enhancement)

    # 폭풍우
    def _effect_storm(self, position, enhancement):
        # '폭풍우' 카드: 시전 위치를 중심으로 세로 일렬 범위 모든 타일 타격
        for i in range(self.layout.shape[0]):
            destroy_prob = 1.0 - 0.15 * abs(i - position[0])
            if random.random() < destroy_prob or enhancement > 0:
                self._destroy_tile((i, position[1]), enhancement)

    # 대폭발
    def _effect_explosion(self, position, enhancement):
        # '대폭발' 카드: 시전 위치 중심으로 X 자 형태 모든 타일 타격
        for offset in range(-self.layout.shape[0], self.layout.shape[0]):
            destroy_prob = 1.0 - 0.15 * abs(offset)
            if random.random() < destroy_prob or enhancement > 0:
                if 0 <= position[0] + offset < self.layout.shape[0]:
                    self._destroy_tile((position[0] + offset, position[1] + offset), enhancement)
                    self._destroy_tile((position[0] + offset, position[1] - offset), enhancement)

    # 정화
    def _effect_purification(self, position, enhancement):
        # '정화' 카드: 시전 위치 중심으로 좌우 타일 타격
        special = 1
        center_x, center_y = position
        self._destroy_tile((center_x, center_y), enhancement, special)  # 중심 타일은 항상 파괴
        for offset in [-1, 1]:
            if 0 <= position[1] + offset < self.layout.shape[1]:
                if random.random() < 0.5 or enhancement > 0:  # 좌우 타일은 50% 확률로 or 강화되면 파괴
                    self._destroy_tile((position[0], position[1] + offset), enhancement, special)
            if enhancement == 2:
                if 0 <= position[0] + offset < self.layout.shape[0]:
                    self._destroy_tile((position[0] + offset, position[1]), enhancement, special)

    # 벼락
    def _effect_lightning(self, position, enhancement):
        # '벼락' 카드: 시전 위치 100% 파괴, 랜덤으로 0~2칸 추가 파괴
        self._destroy_tile(position)

        all_tiles = [(i, j) for i in range(self.layout.shape[0])
                     for j in range(self.layout.shape[1])
                     if self.layout[i, j] != 2 and self.layout[i, j] != 0]

        num_tiles_to_destroy = random.randint(0, min(2 + 2 * enhancement, len(all_tiles)))

        for _ in range(num_tiles_to_destroy):
            tile_to_destroy = random.choice(all_tiles)
            self._destroy_tile(tile_to_destroy)
            all_tiles.remove(tile_to_destroy)  # 파괴된 타일은 리스트에서 제거

        # 랜덤 위치에 랜덤으로 0~1칸 정상 타일 생성
        self._create_random_tiles(random.randint(0, 1))

    # 업화
    def _effect_upfire(self, position, enhancement):
        # '업화' 카드: 시전 위치 중심으로 마름모 형태 2칸 범위 내 파괴, 시전 위치 100% 파괴
        self._destroy_tile(position, enhancement)
        offsets = [(-2, 0), (-1, -1), (-1, 0), (-1, 1), (0, -2), (0, -1), (0, 1), (0, 2), (1, -1), (1, 0), (1, 1), (2, 0)]
        for dx, dy in offsets:
            new_position = (position[0] + dx, position[1] + dy)
            if self._is_within_bounds(new_position) and random.random() < 0.5:
                if random.random() < 0.5 or enhancement > 0:
                    self._destroy_tile(new_position, enhancement)

    # 분출
    def _effect_eruption(self, position, enhancement = 0):
        # '분출' 카드: 시전 위치 100% 파괴
        self._destroy_tile(position)

    # 세계수의 공명
    def _effect_world_tree_resonance(self, position, enhancement = 0):
        # '세계수의 공명' 카드: 시전 위치 중심으로 십자형 타일 2칸 범위 100% 파괴
        special = 1
        offsets = [(0, 0), (-2, 0), (-1, 0), (1, 0), (2, 0), (0, -2), (0, -1), (0, 1), (0, 2)]
        for dx, dy in offsets:
            new_position = (position[0] + dx, position[1] + dy)
            if self._is_within_bounds(new_position):
                self._destroy_tile(new_position, enhancement, special)

    # 특수 타일의 효과를 적용하는 메소드들
    def _effect_rearrange(self):
        # '재배치' 효과: 남은 모든 타일을 무작위 위치로 재배치
        tiles = [(x, y, self.layout[x, y]) for x in range(self.layout.shape[0])
                 for y in range(self.layout.shape[1]) if self.layout[x, y] != -1]

        # 비정상 위치를 제외한 나머지 타일 값만 추출
        tile_values = [value for _, _, value in tiles if value != -1]

        # 추출한 타일 값을 무작위로 섞음
        random.shuffle(tile_values)

        # 무작위로 섞은 타일 값을 다시 게임 보드에 배치
        for (x, y, _), value in zip(tiles, tile_values):
            self.layout[x, y] = value

    def _effect_blessing(self):
        # '축복' 효과: 이번 턴의 카드 사용 횟수를 증가시키지 않음
        self.cards_used -= 1

    def _effect_extra(self):
        # '추가' 효과: 최대 카드 교체 횟수를 하나 늘림
        self.max_swaps += 1

    def _effect_enhance(self):
        # '강화' 효과: 패에 있는 카드를 1회 강화
        if self.hand_enhancements[1 - self.used_card_index] < 2:
            self.hand_enhancements[1 - self.used_card_index] += 1

    def _effect_clone(self):
        # '복제' 효과: 마지막으로 사용한 카드로 교체
        self.hand[1 - self.used_card_index] = self.hand[1 - self.used_card_index]
        self.hand_enhancements[1 - self.used_card_index] = self.hand_enhancements[1 - self.used_card_index]
    def _effect_mystery(self):
        # '신비' 효과: 랜덤하게 '세계수의 공명' 또는 '분출'로 교체
        self.hand[1 - self.used_card_index] = 9 if random.random() < 0.5 else 8

    def _is_game_over(self):
        # 정상 타일(1)과 특수 타일(3~9)이 없는지 확인
        normal_tiles_gone = np.all((self.layout != 1) & (self.layout < 3))
        # 카드 사용 및 교체 횟수가 규정 내인지 확인
        within_card_usage = self.cards_used <= self.max_card_usage
        within_swap_count = self.swap_count <= self.max_swap

        if normal_tiles_gone and within_card_usage and within_swap_count:
            return True, "Win"
        elif not within_card_usage or not within_swap_count:
            return True, "Lose"
        else:
            return False, ""

    def render(self):
        # 환경 상태 출력
        print("Layout:")
        print(self.layout)
        print("Hand: ", self.hand)
        print("Reserve: ", self.reserve)
        print("Cards used: ", self.cards_used)
        print("Swaps used: ", self.swap_count)
        print("Enhancements: ", self.hand_enhancements)


